<a href="https://colab.research.google.com/github/mrchapagain/food_ontology/blob/main/Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here i try to explore different dataset to look if they can be connected, linked or somehow can talk to eachother.**
*   *Data by Frederik*
*   *Frida dataset of Nutrient content of food items*
*   *Reciept from COOP store*
*   *Reciept from my personal Storebox account*
*   *link from TradeSync data from GS1*
*   *Climate Data from Concito*

In [80]:
# Necessary libaries
import pandas as pd
import os

# Data by Frederik

In [238]:
#Import data
path_iss_data="https://github.com/mrchapagain/food_ontology/raw/main/cpd_summary.csv"

#Open data as csv file
data_iss= pd.read_csv(path_iss_data, sep= ";", index_col=False, encoding='latin-1')
data_iss.head(10)

,itemname,itemnumber,category,percent_purchases
0,PANT,8880171,845,"3,369"
1,PANT,8880171,NaN,"2,055"
2,PANT,8880172,845,"1,497"
3,Diesel,4,NaN,"1,126"
4,FISKEAFDELING,209999,NaN,"1,06"
5,Blyfri 95,2,NaN,"0,878"
6,PANT,8880173,845,"0,757"
7,Blyfri 95,1,NaN,"0,74"
8,PANT,8880172,NaN,"0,677"
9,PANT,8880173,NaN,"0,655"


# Frida dataset

In [227]:
# Import data
path_frida_data="https://github.com/mrchapagain/food_ontology/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head()

,Unnamed: 1,Unnamed: 2,Svind,"Energi, kJ","Energi, kcal",Nitrogen-til-protein faktor,"Nitrogen, total","Protein, videnskabelig","Protein, deklaration","Kulhydrat, difference",...,FCF,Alkohol,Aske,Tørstof,Vand,A-vitamin,Retinol,beta-caroten,D-vitamin,D3-vitamin
Nummer,gruppe,Navn,%,kJ,kcal,-,g,g,g,g,...,-,g,g,g,g,RE,µg,µg,µg,µg
1,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
2,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
3,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
4,Rod- og knoldgrøntsager,"Kartoffel, uspec., rå",25,326,77,6.25,0.324,2,2,17.3,...,0.8,0,0.9,20.5,79.5,0.833,0,10,0,0


In [241]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 
               'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)

# Assign columns name that defined just above
data_frida.columns= column_names

# lets see somerowsof the data
data_frida.head()

,Gruppe,Navn,Svind_%,Energy_kj,Energy_kcal,Nitrogen_til_protein_faktor,Nitrogen_total_g,Protein_videnskabelign_g,Protein_deklaration_g,Kulhydrat_difference_g,...,FCF,Alkohol_g,Aske_g,Tørstof_g,Vand_g,A_vitamin_RE,Retinol_µg,beta_caroten_µg,D_vitamin_µg,D3_vitamin_µg
0,Bærfrugt,"Jordbær, rå",4,162,38,6.25,0.106,0.7,0.7,8.3,...,0.8,0,0.4,10,90,3.33,0,40,0,0
1,Kernefrugt,"Æble, uspec., råt",10,233,55,6.25,0.043,0.3,0.3,14.3,...,0.8,0,0.3,15.1,84.9,2.08,0,25,0,0
2,Sydfrugt,"Banan, rå",41,396,93,6.25,0.183,1.1,1.1,22.6,...,0.73,0,0.7,24.6,75.3,4.42,0,53,0,0
3,Rod- og knoldgrøntsager,"Kartoffel, uspec., rå",25,326,77,6.25,0.324,2,2,17.3,...,0.8,0,0.9,20.5,79.5,0.833,0,10,0,0
4,Bærfrugt,"Hindbær, rå",0,215,51,6.25,0.219,1.4,1.4,10.6,...,0.8,0,0.5,13.8,86.2,3.5,0,42,0,0


In [245]:
# To see NAN value and data types of each columns
data_frida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186 entries, 0 to 1185
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Gruppe                       1186 non-null   object
 1   Navn                         1186 non-null   object
 2   Svind_%                      1186 non-null   object
 3   Energy_kj                    1186 non-null   object
 4   Energy_kcal                  1186 non-null   object
 5   Nitrogen_til_protein_faktor  1186 non-null   object
 6   Nitrogen_total_g             1186 non-null   object
 7   Protein_videnskabelign_g     1186 non-null   object
 8   Protein_deklaration_g        1186 non-null   object
 9   Kulhydrat_difference_g       1186 non-null   object
 10  Kulhydrat_tilgængelig_g      1186 non-null   object
 11  Kulhydrat_deklaration_g      1186 non-null   object
 12  Tilsat_sukker_g              1186 non-null   object
 13  Kostfibre_g                  1186

***Above dataset do not contain NAN value and all of the columns are object type, so someof the columns need to be changed to interger or float data type to work as numbers value***

# Reciept from COOP store

In [85]:
path="https://github.com/mrchapagain/food_ontology/raw/main/cpd_summary.csv"

data= pd.read_csv(path, sep= ";", index_col=0, encoding='latin-1')
data.index.unique()

Index([                'PANT',               'Diesel',        'FISKEAFDELING',
                  'Blyfri 95',    'ØKO BANANER, LØSE',         'BANANER,LØSE',
              'Blyfri 95 E10',              'BANANER',           'FLASKEPANT',
            'BETJENT FISK KG',
       ...
       'AAS MUNDBIND TYPE II',      '3F-HYLDEKNÆGT M',       'SKAGERAK BØJLE',
           'SMÅUDB. U/100 KR', 'SODASTREAM OMBYTNING',  'TECHNIC 42073 BASH!',
             'TEKSTILLEDNING', 'PINOTEX SUPERDEC SOR',  'UKRUDTSDUG BASIC 1X',
            'BR MÆRKER RETUR'],
      dtype='object', name='itemname', length=65449)

# TradeSync data from GS1 

# Recipt from my personal Storebox account

# Climate Data from Concito

In [254]:
#Import data
path_climate_data="https://github.com/mrchapagain/food_ontology/raw/main/ClimateData.xlsx"

# Loading the data into the data-frame
col_names= ["id", "Product_dk", "Category_dk", "Product_en", "Category_en", "Unit", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg", "Energy_KJ", "Fat_g", "Carb_g", "Protein_g", "Data_Source", "Comments", "GPC_Level4_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_Category_en", "GPC_Category_dk", "GPC_Level4_dk", "product_type", "GPC_level1", "Food_group", "GPC_level2", "Un/Processed", "GPC_Level3", "Extra_category"]

# Original columns name are replaced with ralavant names (col_names)
data_climate= pd.read_excel(path_climate_data, sheet_name=1, index_col=False, header=None, skiprows=1, names= col_names).round(decimals = 2)
data_climate.head(3)

/usr/local/lib/python3.7/dist-packages/pandas/io/excel/_base.py:1292: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  **kwds,


,id,Product_dk,Category_dk,Product_en,Category_en,Unit,Agriculture,iLUC,Processing,Packaging,...,GPC_Category_en,GPC_Category_dk,GPC_Level4_dk,product_type,GPC_level1,Food_group,GPC_level2,Un/Processed,GPC_Level3,Extra_category
0,Ra00001,"Peberfrugt, rød, rå",Grøntsager,"Pepper, sweet, red, raw",Vegetables,kg,0.25,0.03,0.0,0.14,...,Vegetables,Grøntsager,10006191,Square Tapering Sweet Peppers (Peg Top),50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260400.0,Peppers
1,Ra00002,"Tomat, uspec., rå",Grøntsager,"Tomato, ripe, raw, origin unknown",Vegetables,kg,0.07,0.01,0.0,0.14,...,Vegetables,Grøntsager,10006165,Tomatoes – Round,50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260300.0,Tomatoes
2,Ra00003,"Squash, rå",Grøntsager,"Squash, raw",Vegetables,kg,0.14,0.04,0.0,0.06,...,Vegetables,Grøntsager,10006355,Squash (Calabaza),50000000.0,Food/Beverage/Tobacco,50260000.0,Vegetables (Non Leaf) – Unprepared/Unprocessed...,50260700.0,Cucurbits – Edible Peel


***The reason to open this climate dataset is to see if it is possible link other dataset with column name "GPC_Level" which is releted to GS1 Trade Sync code.***

**Let´s have a look dataset with only selected columns that might make sense\**

In [257]:
cols_to_use= ["Product_en", "Category_en", "ID_Food", "ID_Pack", "ID_Retail", "GPC_level1", "GPC_level2", "GPC_Level3", "GPC_Level4_dk", "Fat_g", "Carb_g", "Protein_g", "Energy_KJ", "Agriculture", "iLUC", "Processing", "Packaging", "Transport", "Retail", "Total_CO2_eq/kg"]

data_climate_selected= data_climate.loc[:, cols_to_use].round(decimals = 2)
data_climate_selected.head()

,Product_en,Category_en,ID_Food,ID_Pack,ID_Retail,GPC_level1,GPC_level2,GPC_Level3,GPC_Level4_dk,Fat_g,Carb_g,Protein_g,Energy_KJ,Agriculture,iLUC,Processing,Packaging,Transport,Retail,Total_CO2_eq/kg
0,"Pepper, sweet, red, raw",Vegetables,Cm039,Pa059,Ma0332,50000000.0,50260000.0,50260400.0,10006191,0.1,5.2,0.9,133.0,0.25,0.03,0.0,0.14,0.58,0.01,1.02
1,"Tomato, ripe, raw, origin unknown",Vegetables,Cm167,Pa059,Ma0332,50000000.0,50260000.0,50260300.0,10006165,0.1,3.2,0.8,87.0,0.07,0.01,0.0,0.14,0.48,0.01,0.70
2,"Squash, raw",Vegetables,Cm135,Pa040,Ma0332,50000000.0,50260000.0,50260700.0,10006355,0.2,2.4,1.4,81.0,0.14,0.04,0.0,0.06,0.58,0.01,0.83
3,"Aubergine, raw",Vegetables,Cm056,Pa059,Ma0332,50000000.0,50260000.0,50260500.0,10006128,0.1,3.1,0.9,83.0,0.16,0.02,0.0,0.14,0.65,0.01,0.97
4,"Tomato, dried",Vegetables,Fa0152,Pa066,Ma0332,50000000.0,50100000.0,50102100.0,10000272,3.0,43.5,14.1,1187.0,0.91,0.09,-0.0,0.26,0.68,0.01,1.95
